In [3]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode
'''
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

dtype = torch.FloatTensor
predict_length=12
sentences = []

#Read data from txt
with open('quote.txt','r',encoding='utf-8') as file:
    for eline in file.readlines():
        eline=eline.strip()
        if len(eline.split())>=predict_length+1 :
            sentences.append(eline)
            
word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

# TextRNN Parameter
batch_size = len(sentences)
n_hidden = 32 # number of hidden units in one cell

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[0:predict_length]]
        input_batch.append(np.eye(n_class)[input])
        
    #special order
    #[step,batch]->[step*batch]
    for i in range(1,predict_length+1):
        for sen in sentences:
            word = sen.split()
            target_batch.append(word_dict[word[i]])
    return input_batch, target_batch

# to Torch.Tensor
input_batch, target_batch = make_batch(sentences)
input_batch = Variable(torch.Tensor(input_batch)).cuda()
target_batch = Variable(torch.LongTensor(target_batch)).cuda()
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()

        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))

    def forward(self, hidden, X):
        X = X.transpose(0, 1) # X : [n_step, batch_size, n_class]
        
        outputs, hidden = self.rnn(X, hidden)
        outputs=outputs.view(-1,n_hidden)
        model = torch.mm(outputs, self.W) + self.b # model : [batch_size,step, n_class]
        #[step*batch_size,n_class]
        return model

model = TextRNN()
model = model.cuda()
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(20000):
    optimizer.zero_grad()

    # hidden : [num_layers * num_directions, batch, hidden_size]
    hidden = Variable(torch.zeros(1, batch_size, n_hidden)).cuda()
    # input_batch : [batch_size, n_step, n_class]
    output = model(hidden, input_batch)

    
    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print(target_batch)
        print(output)
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

input = [sen.split()[:2] for sen in sentences]

# Predict

# print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

tensor([2532, 2117, 1553,  ..., 2246,   28,  379], device='cuda:0')
tensor([[ -1.4877,  -0.9047,  -2.8960,  ...,   0.3430,   1.4598,  -1.8060],
        [ -1.0708,  -1.4260,   0.4693,  ...,   2.3263,   3.9504,  -1.0496],
        [ -0.2043,  -0.0211,  -2.9999,  ...,  -0.9497,  -1.0378,  -1.7619],
        ...,
        [ -3.4578,  -1.2809,  -5.3209,  ...,  -9.9956,  -6.7028,  -1.1804],
        [ -2.8157,   2.6809,  -6.6581,  ...,  -6.6023,  -2.1903,  -2.8498],
        [ -2.4329,   0.1383,   0.4161,  ..., -10.5359,  -1.2644,  -0.0631]],
       device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 1000 cost = 0.243396
tensor([2532, 2117, 1553,  ..., 2246,   28,  379], device='cuda:0')
tensor([[ -1.6973,  -0.9967,  -3.2603,  ...,   0.1126,   1.7666,  -1.9408],
        [ -0.9001,  -1.3635,   1.8028,  ...,   3.6035,   3.9002,  -0.8291],
        [ -0.2692,  -0.0419,  -3.4160,  ...,  -0.7551,  -1.5119,  -1.6412],
        ...,
        [ -3.2794,  -1.6370,  -6.6835,  ...,  -9.9379,  -8.1285,  -0.6484],
 

       device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 13000 cost = 0.135890
tensor([2532, 2117, 1553,  ..., 2246,   28,  379], device='cuda:0')
tensor([[ -1.8732,  -1.0430,  -1.0286,  ...,  -0.2952,   2.7070,  -2.3844],
        [ -0.7500,  -1.2791,   6.7583,  ...,   4.0659,   3.6227,  -0.7519],
        [ -0.6163,  -0.1547,  -5.1186,  ...,  -0.8077,  -2.0093,  -1.5871],
        ...,
        [ -2.8866,  -1.7406,  -9.4215,  ...,  -9.8494, -11.8864,  -0.7505],
        [ -1.8132,   0.7006,  -6.3873,  ...,  -7.4189,  -7.1330,  -0.7617],
        [ -2.1337,   0.6861,   6.8882,  ..., -10.9599,  -0.3112,  -0.2352]],
       device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 14000 cost = 0.135870
tensor([2532, 2117, 1553,  ..., 2246,   28,  379], device='cuda:0')
tensor([[ -1.8683,  -1.0525,  -0.8297,  ...,  -0.2855,   2.7495,  -2.3806],
        [ -0.7485,  -1.2838,   7.1317,  ...,   4.1006,   3.6377,  -0.7500],
        [ -0.6313,  -0.1562,  -5.2311,  ...,  -0.7927,  -2.0391,  -1.5780],
        ...,

In [9]:
hidden = Variable(torch.zeros(1, 1, n_hidden)).cuda()
test_input="You"#放在开头的词
words_input=test_input.split()
num_input = [word_dict[n] for n in words_input]
for i in range(predict_length):
    test_batch=[]
    test_batch.append(np.eye(n_class)[num_input])
    test_batch=Variable(torch.Tensor(test_batch)).cuda()
    predict = model(hidden, test_batch)
    num_input.append(int(predict.argmax(1)[-1]))
for i in num_input:
    print(number_dict[i])


You
don't
have
to
tell
even
a
third
generation
American
about
the
Mafia
